# Add Info to JSON Files
## Authors: Ganesh Arvapalli and Patrick Myers

This file loads in a JSON file and the corresponding nifti image. Important information about the image is added to the JSON file

In [1]:
import os
from os.path import isfile, join
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import json
import pdb
import glob
from nilearn import plotting as nip
from nilearn import image

## Adds the size of each parcel in an atlas to that JSON file

Flattens the nifti image into a 1D array. Then puts the data into a 2D list where each row, i, contains the locations of the voxels in the ith parcel. The size of each of these lists is calculated. This information is added to the corresponding key in the JSON file

In [2]:
def Size_JSON(json_file, nifti_file, outname):    
    img = nib.load(nifti_file)
    img_data = img.get_fdata()
    img_flat = img_data.flatten()
    img_flat.tolist()
    img_flat = [int(i) for i in img_flat]
    values = np.unique(img_flat)
    counts = np.array(range(1, len(img_flat)+1))
    counts = [int(i) for i in counts]
    img_flatten = list(zip(img_flat, counts))
    img_flatten = sorted(img_flatten, key = lambda l:l[0])

    img_cell = [[] for i in range(len(values))]
    start = 0
    for i in range(0,len(values)):
        img_list = [x[0] for x in img_flatten]
        end = max(loc for loc, val in enumerate(img_list) if val == values[i])
        for j in range(start,end):
            img_cell[i].append(img_flatten[j][1])  
        start = end+1
    
    with open(json_file) as f:
        data = json.load(f)
    
    for i in range(0,len(values)):
        data[str(i)]["size"] = str(len(img_cell[i]))
    
    with open(outname, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=True)


### Run for every JSON/Nifti pair

Not every Nifti file has a corresponding JSON pair, so the JSON files were used as the key in the search for the corresponding Nifti file. Specifies the output JSON to a new folder to not get confused.

In [16]:
os.chdir('../JSON_files')
JSON_files = [f for f in os.listdir('.') if os.path.isfile(f)]
files = [f.split('.')[0] for f in JSON_files]
files = [i for i in files if not i.startswith("DS")]
files

['aal_space-MNI152NLin6_res-1x1x1',
 'brodmann_space-MNI152NLin6_res-1x1x1',
 'CPAC200_space-MNI152NLin6_res-1x1x1',
 'desikan_space-MNI152NLin6_res-1x1x1',
 'DKT_space-MNI152NLin6_res-1x1x1',
 'glasser_space-MNI152NLin6_res-1x1x1',
 'HarvardOxfordcort-maxprob-thr25_space-MNI152NLin6_res-1x1x1',
 'HarvardOxfordsub-maxprob-thr25_space-MNI152NLin6_res-1x1x1',
 'hemispheric_space-MNI152NLin6_res-1x1x1',
 'JHU_space-MNI152NLin6_res-1x1x1',
 'princetonvisual-top_space-MNI152NLin6_res-1x1x1',
 'slab1068_space-MNI152NLin6_res-1x1x1',
 'slab907_space-MNI152NLin6_res-1x1x1',
 'Talairach_space-MNI152NLin6_res-1x1x1',
 'tissue_space-MNI152NLin6_res-1x1x1',
 'yeo-17_space-MNI152NLin6_res-1x1x1',
 'yeo-7-liberal_space-MNI152NLin6_res-1x1x1',
 'yeo-7_space-MNI152NLin6_res-1x1x1']

In [3]:
os.chdir('../JSON_files')
JSON_files = [f for f in os.listdir('.') if os.path.isfile(f)]
files = [f.split('.')[0] for f in JSON_files]
files = [i for i in files if not i.startswith("DS")]
JSON_path = '../JSON_files/'
JSON_extension = '.json'
Nifti_path = '../Nifti_images/'
Nifti_extension = '.nii.gz'
outfile_path = '../Updated_JSON_files/'

for file in files:
    json_file = JSON_path+file+JSON_extension
    print(json_file)
    nifti_file = Nifti_path+file+Nifti_extension
    outfile = outfile_path+file+'_updated'+JSON_extension
    Size_JSON(json_file,nifti_file,outfile)




../JSON_files/aal_space-MNI152NLin6_res-1x1x1.json
../JSON_files/brodmann_space-MNI152NLin6_res-1x1x1.json
../JSON_files/CPAC200_space-MNI152NLin6_res-1x1x1.json
../JSON_files/desikan_space-MNI152NLin6_res-1x1x1.json
../JSON_files/DKT_space-MNI152NLin6_res-1x1x1.json
../JSON_files/glasser_space-MNI152NLin6_res-1x1x1.json
../JSON_files/HarvardOxfordcort-maxprob-thr25_space-MNI152NLin6_res-1x1x1.json
../JSON_files/HarvardOxfordsub-maxprob-thr25_space-MNI152NLin6_res-1x1x1.json
../JSON_files/hemispheric_space-MNI152NLin6_res-1x1x1.json
../JSON_files/JHU_space-MNI152NLin6_res-1x1x1.json
../JSON_files/princetonvisual-top_space-MNI152NLin6_res-1x1x1.json
../JSON_files/slab1068_space-MNI152NLin6_res-1x1x1.json
../JSON_files/slab907_space-MNI152NLin6_res-1x1x1.json
../JSON_files/Talairach_space-MNI152NLin6_res-1x1x1.json
../JSON_files/tissue_space-MNI152NLin6_res-1x1x1.json
../JSON_files/yeo-17_space-MNI152NLin6_res-1x1x1.json
../JSON_files/yeo-7-liberal_space-MNI152NLin6_res-1x1x1.json
../JSO

In [19]:
def get_centers(brain):
    """
    Get coordinate centers given a nifti image loaded with nibabel
    
    Returns a dictionary of label: coordinate as an [x, y, z] array
    """
    dat = brain.get_data()
    labs = np.unique(dat)
    labs = labs[labs != 0]
    print('here')
    fd_dat = np.stack([np.asarray(dat == lab).astype('float64') for lab in labs], axis=3)
    print('here2')
    parcels = nib.Nifti1Image(dataobj=fd_dat, header=brain.header, affine=brain.affine)
    regions_imgs = image.iter_img(parcels)
    # compute the centers of mass for each ROI
    coords_connectome = [nip.find_xyz_cut_coords(img) for img in regions_imgs]
    return dict(zip(labs, coords_connectome)), parcels

In [18]:
def main():
    labdir = '../Nifti_images/Test'
    specparc = ['desikan', 'tissue', 'DK', 'pp264']
    #specdir = './label'
    jsdir = '../JSON_files/Test'
    #outdir = './label/label_updated2'
    brainglob = glob.glob(os.path.join(labdir, '*.nii.gz'))
    jsonglob = glob.glob(os.path.join(jsdir, '*.json'))
    # iterate over the brains
    for brainf in brainglob:
        # get the name of the particular parcel
        brain_name = str.split(os.path.basename(brainf), '.')[0]
        bname = str.split(brain_name, '_')[0]
        #jsout = os.path.join(outdir, "{}.json".format(brain_name))
        print(brainf)
        parcel_im = nib.load(brainf)
        parcel_centers, parcels = get_centers(parcel_im)
    return parcel_centers, parcels

In [21]:
centers, parcels = main()

../Nifti_images/Test\yeo-7_space-MNI152NLin6_res-1x1x1.nii.gz
here
here2


['ImageArrayProxy',
 'ImageSlicer',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_affine',
 '_affine2header',
 '_compressed_suffixes',
 '_data',
 '_data_cache',
 '_dataobj',
 '_fdata_cache',
 '_get_fileholders',
 '_header',
 '_meta_sniff_len',
 '_sniff_meta_for',
 'affine',
 'as_reoriented',
 'dataobj',
 'extra',
 'file_map',
 'files_types',
 'filespec_to_file_map',
 'filespec_to_files',
 'from_file_map',
 'from_filename',
 'from_files',
 'from_image',
 'get_affine',
 'get_data',
 'get_data_dtype',
 'get_fdata',
 'get_filename',
 'get_header',
 'get_qform',
 'get_sform',
 'get_shape',
 'header',
 'header_class',
 'in_memory',
 'instance_to_filename'